In [1]:
!pip install gymnasium
!pip install stable_baselines3
!pip install sb3_contrib
!pip install gymnasium[mujoco]
!pip install stable-baselines[mpi]==2.10.0
!pip install swig
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [1]:
import gymnasium as gym
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO
import torch
import os
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold, BaseCallback
import time
from stable_baselines3.common.evaluation import evaluate_policy
from collections import defaultdict

In [2]:
os.chdir('/content/drive/MyDrive/Tilburg/Masters CSAI/Year 1 - Semester 2/Deep RL/Project/models/PPO')

In [3]:
from stable_baselines3.common.evaluation import evaluate_policy
from collections import defaultdict

# # load and evaluate each model
# Code derived from provided assignment PDF and stable_baselines 3 documentation
seeds = [12,33,47]

results = defaultdict(list)
for seed in seeds:
  eval_env = make_vec_env('BipedalWalker-v3', n_envs = 4, seed = 1)
  #env = VecNormalize(env, norm_obs=True, norm_reward = True)
  eval_env = VecNormalize.load(f'PPO_walker_seed_{seed}_24h.pkl',eval_env)
  eval_env.training = False  # Important to disable training mode when evaluating
  eval_env.norm_reward = True
  ev_model = PPO.load(f'PPO_walker_seed_{seed}_24h.zip')
  mean_reward, std_reward = evaluate_policy(ev_model, eval_env, n_eval_episodes=100, deterministic = True)
  print(f'seed : {seed}')
  print(f'mean_reward={mean_reward:.2f} +/- {std_reward}')
  print('-----------')
  #results[m].append([eval_mass,mean_reward,std_reward])

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(


seed : 12
mean_reward=-121.31 +/- 3.5252011101798084
-----------


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at most 16 arguments, got 18
  warnings.warn(


seed : 33
mean_reward=299.90 +/- 67.7871010263029
-----------
seed : 47
mean_reward=312.48 +/- 34.48250365029754
-----------


In [10]:
from stable_baselines3 import PPO

seeds = [12, 33,47]
results = defaultdict(list)

for seed in seeds:
    eval_env = make_vec_env('BipedalWalker-v3', n_envs=1, seed=seed)
    eval_env = VecNormalize.load(f'PPO_walker_seed_{seed}_24h.pkl',eval_env)
    eval_env.training = False  # Disable training mode for evaluation
    eval_env.norm_reward = False

    model = PPO.load(f'PPO_walker_seed_{seed}_24h.zip')

    episode_rewards = []
    episode_lengths = []

    for _ in range(100):  # Evaluate for 100 episodes
        obs = eval_env.reset()
        done = False
        total_rewards = 0
        steps = 0
        while not done and steps < 1600:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, info = eval_env.step(action)
            #print(reward)
            total_rewards += reward
            steps += 1

        episode_rewards.append(total_rewards)
        episode_lengths.append(steps)
        print(f'Seed {seed}: Episode reward: {total_rewards}, Steps: {steps}')

    mean_reward = sum(episode_rewards) / len(episode_rewards)
    std_reward = (sum((x - mean_reward) ** 2 for x in episode_rewards) / len(episode_rewards)) ** 0.5
    mean_steps = sum(episode_lengths) / len(episode_lengths)
    print(f'Seed: {seed}, Mean Reward: {mean_reward} +/- {std_reward}, Mean Steps: {mean_steps}')
    print('-----------')


Seed 12: Episode reward: [-124.416336], Steps: 110
Seed: 12, Mean Reward: [-124.416336] +/- [0.], Mean Steps: 110.0
-----------
Seed 33: Episode reward: [317.58908], Steps: 768
Seed: 33, Mean Reward: [317.58908] +/- [0.], Mean Steps: 768.0
-----------
Seed 47: Episode reward: [317.9514], Steps: 876
Seed: 47, Mean Reward: [317.9514] +/- [0.], Mean Steps: 876.0
-----------
